# SageMaker Demo Cleanup
- Empties the S3 bucket so it can be deleted by CloudFormation

In [ ]:
import json

# Path to the JSON file
file_path = "/opt/ml/metadata/resource-metadata.json"

# Try to open the file and load its content
try:
    with open(file_path, "r") as file:
        metadata = json.load(file)

# Stop running if the file is not found
except FileNotFoundError:
    raise FileNotFoundError(f"The file at {file_path} was not found.")

# Stop running if the file is not JSON
except json.JSONDecodeError:
    raise ValueError("Failed to decode JSON from the file.")

# Stop running if any other errors are encountered
except Exception as e:
    raise RuntimeError(f"An unexpected error occurred: {str(e)}")

# Access the ResourceName value
notebook_instance_name = metadata["ResourceName"]

print("Notebook instance name:", notebook_instance_name)

In [ ]:
import boto3

ssm_client = boto3.client("ssm")

In [ ]:
# Read value from Parameter Store
def read_from_parameter_store(name):
    response = ssm_client.get_parameter(Name=name, WithDecryption=True)
    return response["Parameter"]["Value"]


# Get the S3 bucket name and region from Parameter Store
bucket_name = read_from_parameter_store(f"/{notebook_instance_name}/s3bucket")
region = read_from_parameter_store(f"/{notebook_instance_name}/region")

print(f"\tS3 Bucket Name from ParameterStore: {bucket_name}")
print(f"\tRegion Name from ParameterStore: {region}")

In [ ]:
!aws sts get-caller-identity

In [ ]:
!aws s3 rm s3://{bucket_name} --recursive

In [ ]:
from IPython.display import display, Markdown

markdown_text = f"""
## S3 Bucket has been emptied
Proceed with decommissioning the lab by deleteing the CloudFormation stack

## [CloudFormation Stacks](https://{region}.console.aws.amazon.com/cloudformation/home)
"""

display(Markdown(markdown_text))